In [1]:
import pandas as pd
import os
from datetime import datetime as dt

import itertools
import networkx as nx
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from collections import Counter

import copy

base = os.getcwd() + "\\"

In [2]:
matches_raw =      pd.read_excel(base + "IPL_2024.xlsx", sheet_name = 'League Matches')
points_table_raw = pd.read_excel(base + "IPL_2024.xlsx", sheet_name = 'Points Table')
mapping      = pd.read_excel(base + "IPL_2024.xlsx", sheet_name = 'Mapping')

display(mapping)

,Team Full Name,Team Half Name,Team Abbr
0,Chennai Super Kings,Super Kings,CSK
1,Punjab Kings,Kings,PBKS
2,Kolkata Knight Riders,Knight Riders,KKR
3,Rajasthan Royals,Royals,RR
4,Gujarat Titans,Titans,GT
5,Royal Challengers Bengaluru,Royal Challengers,RCB
6,Sunrisers Hyderabad,Sunrisers,SRH
7,Lucknow Super Giants,Super Giants,LSG
8,Delhi Capitals,Capitals,DC
9,Mumbai Indians,Indians,MI


In [3]:
matches = matches_raw.copy()
matches.columns = [i.replace("\n"," ") for i in matches.columns]
matches['DATETIME'] = pd.to_datetime(matches['DATE'] + " " + matches['TIME'])

matches = matches.merge(mapping[['Team Full Name','Team Abbr']].rename(columns = {'Team Full Name':'HOME', 'Team Abbr':'HOME_TEAM'}),on = 'HOME', how = 'left')
matches = matches.merge(mapping[['Team Full Name','Team Abbr']].rename(columns = {'Team Full Name':'AWAY', 'Team Abbr':'AWAY_TEAM'}),on = 'AWAY', how = 'left')

curr_date = dt(2024,5,5,19)
curr_date = dt.today()
# .replace(hour = 0)

print(curr_date)

future_matches = matches[matches['DATETIME']>=curr_date]
display(future_matches)

2024-05-08 22:25:12.591402


,MATCH NO,MATCH DAY,DATE,DAY,TIME,HOME,AWAY,STADIUM,DATETIME,HOME_TEAM,AWAY_TEAM
57,58,49,09-May-24,Thu,7:30PM,Punjab Kings,Royal Challengers Bengaluru,Dharamshala,2024-05-09 19:30:00,PBKS,RCB
58,59,50,10-May-24,Fri,7:30PM,Gujarat Titans,Chennai Super Kings,Ahmedabad,2024-05-10 19:30:00,GT,CSK
59,60,51,11-May-24,Sat,7:30PM,Kolkata Knight Riders,Mumbai Indians,Kolkata,2024-05-11 19:30:00,KKR,MI
60,61,52,12-May-24,Sun,3:30PM,Chennai Super Kings,Rajasthan Royals,Chennai,2024-05-12 15:30:00,CSK,RR
61,62,52,12-May-24,Sun,7:30PM,Royal Challengers Bengaluru,Delhi Capitals,Bengaluru,2024-05-12 19:30:00,RCB,DC
62,63,53,13-May-24,Mon,7:30PM,Gujarat Titans,Kolkata Knight Riders,Ahmedabad,2024-05-13 19:30:00,GT,KKR
63,64,54,14-May-24,Tue,7:30PM,Delhi Capitals,Lucknow Super Giants,Delhi,2024-05-14 19:30:00,DC,LSG
64,65,55,15-May-24,Wed,7:30PM,Rajasthan Royals,Punjab Kings,Guwahati,2024-05-15 19:30:00,RR,PBKS
65,66,56,16-May-24,Thu,7:30PM,Sunrisers Hyderabad,Gujarat Titans,Hyderabad,2024-05-16 19:30:00,SRH,GT
66,67,57,17-May-24,Fri,7:30PM,Mumbai Indians,Lucknow Super Giants,Mumbai,2024-05-17 19:30:00,MI,LSG


In [4]:
points_table = points_table_raw.copy()
points_table = points_table.merge(mapping[['Team Half Name','Team Abbr']].rename(columns = {'Team Half Name':'TEAM', 'Team Abbr':'TEAM_NAME'}),on = 'TEAM', how = 'left')
display(points_table)

# points_table.loc[points_table['TEAM_NAME']=='SRH', 'PTS']+=2
# display(points_table)


,TEAM,MATCHES,WON,LOST,PTS,NRR,TEAM_NAME
0,Knight Riders,11,8,3,16,1.453,KKR
1,Royals,11,8,3,16,0.476,RR
2,Sunrisers,12,7,5,14,0.406,SRH
3,Super Kings,11,6,5,12,0.700,CSK
4,Capitals,12,6,6,12,-0.316,DC
5,Super Giants,12,6,6,12,-0.769,LSG
6,Royal Challengers,11,4,7,8,-0.049,RCB
7,Kings,11,4,7,8,-0.187,PBKS
8,Indians,12,4,8,8,-0.212,MI
9,Titans,11,4,7,8,-1.320,GT


In [5]:
class Team:
    def __init__(self, name, points=0):
        self.name = name
        self.points = points

class Match:
    def __init__(self, team1, team2):
        self.team1 = team1
        self.team2 = team2

team_pts = points_table[['TEAM_NAME','PTS']].set_index('TEAM_NAME').to_dict()['PTS']
teams = {t:Team(t,p) for t,p in team_pts.items()}

In [6]:
future_match_dict = future_matches.set_index('MATCH NO').T.to_dict()
match_dict = {k:Match(teams[v['HOME_TEAM']], teams[v['AWAY_TEAM']]) for k,v in future_match_dict.items()}

In [7]:
print(list([i.name for i in teams.values()]), list(match_dict.values()))


['KKR', 'RR', 'SRH', 'CSK', 'DC', 'LSG', 'RCB', 'PBKS', 'MI', 'GT'] [<__main__.Match object at 0x00000164EC93BAF0>, <__main__.Match object at 0x00000164EC93BD00>, <__main__.Match object at 0x00000164EC93BD60>, <__main__.Match object at 0x00000164EC93B550>, <__main__.Match object at 0x00000164EC93B7F0>, <__main__.Match object at 0x00000164EC994970>, <__main__.Match object at 0x00000164EC9944C0>, <__main__.Match object at 0x00000164EC958730>, <__main__.Match object at 0x00000164EC958B80>, <__main__.Match object at 0x00000164EC958940>, <__main__.Match object at 0x00000164EC9585B0>, <__main__.Match object at 0x00000164EC958DF0>, <__main__.Match object at 0x00000164EC9589A0>]


In [8]:
class MatchOutcome:
    def __init__(self, match, result, winners = [], parent=None):
        self.match = match
        self.result = result
        self.parent = parent
        self.children = []
        self.winners = winners
        # self.width = len(self.children)

        # if parent:
        #     self.width = result + 1
            
        #     self.level = self.parent.level + 1
        # else:
        #     self.level = 0
        #     self.width = result

        # print(self.level, self.width)

        # if match:
        #     print(f"{match.team1.name} v {match.team2.name} \t W - ",[match.team1.name,match.team2.name][result])
        #     if result==0: pt_tbl[match.team1.name]+=2
        #     else: pt_tbl[match.team2.name]+=2



        # print(sorted(tuple(pt_tbl.items()), key = lambda x: -x[1]))
        # self.pt_tbl = pt_tbl

def generate_match_tree(matches):
    root = MatchOutcome(None, None)
    generate_children(root, matches)
    return root

def generate_children(parent_node, remaining_matches):
    if not remaining_matches:
        return
    
    match = remaining_matches[0]
    for result in [0, 1]: 
        print(parent_node.winners)
        child_node = MatchOutcome(match, result,  parent_node.winners + [[match.team1.name,match.team2.name][result]], parent_node)
        parent_node.children.append(child_node)
        generate_children(child_node, remaining_matches[1:])



# print(points_dict)
root = generate_match_tree(list(match_dict.values()))
        

[]
['PBKS']
['PBKS', 'GT']
['PBKS', 'GT', 'KKR']
['PBKS', 'GT', 'KKR', 'CSK']
['PBKS', 'GT', 'KKR', 'CSK', 'RCB']
['PBKS', 'GT', 'KKR', 'CSK', 'RCB', 'GT']
['PBKS', 'GT', 'KKR', 'CSK', 'RCB', 'GT', 'DC']
['PBKS', 'GT', 'KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR']
['PBKS', 'GT', 'KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH']
['PBKS', 'GT', 'KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI']
['PBKS', 'GT', 'KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'RCB']
['PBKS', 'GT', 'KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'RCB', 'SRH']
['PBKS', 'GT', 'KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'RCB', 'SRH']
['PBKS', 'GT', 'KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'RCB']
['PBKS', 'GT', 'KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'RCB', 'PBKS']
['PBKS', 'GT', 'KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'RCB', 'PBKS']
['PBKS', 'GT', 'KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI']
['PBKS', 'GT', 'KKR', 'CSK', 'RCB', 'GT', 'DC', 'RR', 'SRH', 'MI', 'CSK']

In [9]:
def evaluate_condition(node, points_dict, check_team):
    # Example condition: Check if the leaf node's match result is a tie
    winning_pts = dict(Counter(node.winners))
    for t in points_dict.keys():
        if t not in winning_pts.keys():
            winning_pts[t] = 0
        else: winning_pts[t] = winning_pts[t]*2

            
    all_points = list({t: points_dict[t] + winning_pts[t] for t in points_dict.keys()}.items())
    
    all_points = sorted(list(all_points), key = lambda x: -x[-1])
    top4 = all_points[:3]

    if all([t in [i[0] for i in top4] for t in check_team]): 
        print(all_points)
        return True
    return False
    # return node.match is not None and node.match.team1.name == "Team A" and node.result == 1  # Replace with your condition

def prune_tree(root, points_dict, check_team):
    win_list = []
    stack = [root]
    leaf_node = []
    while stack:
        node = stack.pop()
        if not node.children:  # Leaf node
            leaf_node.append(node)
            if not evaluate_condition(node, dict(points_dict), check_team):
                # Prune subtree by removing all children
                node.parent.children.remove(node)
            else:
                win_list.append(node)
        else:
            stack.extend(node.children)
    return root, win_list, leaf_node

points_dict = points_table.set_index('TEAM_NAME').to_dict()['PTS']
pruned_tree, win_leaf, leaf_nodes = prune_tree(copy.deepcopy(root), points_dict, ["RCB"])

In [10]:
print(len(win_leaf)/len(leaf_nodes)*100)

0.0


In [11]:
def evaluate_condition(node, points_dict, check_team):
    # Example condition: Check if the leaf node's match result is a tie
    winning_pts = dict(Counter(node.winners))
    for t in points_dict.keys():
        if t not in winning_pts.keys():
            winning_pts[t] = 0
        else: winning_pts[t] = winning_pts[t]*2

            
    all_points = list({t: points_dict[t] + winning_pts[t] for t in points_dict.keys()}.items())
    
    all_points = [i[0] for i in sorted(list(all_points), key = lambda x: -x[-1])]

    return all_points.index(check_team)+1
    # top4 = all_points[:3]

    # if all([t in [i[0] for i in top4] for t in check_team]): 
    #     print(all_points)
    #     return True
    # return False

def prune_tree(root, points_dict, check_team):
    # print(check_team)
    point_list = []
    stack = [root]
    leaf_node = []
    while stack:
        node = stack.pop()
        if not node.children:  # Leaf node
            leaf_node.append(node)
            point_list.append(evaluate_condition(node, dict(points_dict), check_team))
            # if not evaluate_condition(node, dict(points_dict), check_team):
            #     # Prune subtree by removing all children
            #     node.parent.children.remove(node)
            # else:
            #     win_list.append(node)
        else:
            stack.extend(node.children)

    point_dict = dict(Counter(point_list))
    point_dict = {k:v/len(point_list) for k,v in point_dict.items()}
    return point_dict

points_dict = points_table.set_index('TEAM_NAME').to_dict()['PTS']

team_points = {}
for t in mapping['Team Abbr'].unique():
    print(t)
    team_points[t] =  prune_tree(copy.deepcopy(root), points_dict, t)

CSK
PBKS
KKR
RR
GT
RCB
SRH
LSG
DC
MI


In [31]:
team_points2 = {k:pd.DataFrame(v, index = [k]) for k,v in team_points.items()}
perc = pd.concat(team_points2.values()).T.sort_index().T.rename_axis('RANK')
perc.loc[:,'Top 4'] = perc.iloc[:,:4].sum(axis=1)
# perc.loc['Total',:] = perc.sum(axis=0)
# print(perc.columns)
perc = perc.T[list(perc[['Top 4']].sort_values(by = 'Top 4', ascending = False).index.tolist())]
perc = perc.fillna(0).applymap(lambda x: f'{x:.2%}')



# Define a function to apply conditional formatting
def highlight_cells(val):
    # print(val)
    return f'background-color: rgba(0, 100, 255, {val})'  # Green color with alpha based on value



# Apply styling to the DataFrame
styled_df = perc.style.applymap(highlight_cells)



# Render styled DataFrame
styled_df

Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'Top 4'], dtype='object')


RANK,KKR,RR,SRH,CSK,DC,LSG,RCB,PBKS,MI,GT
1,62.50%,37.50%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
2,33.20%,57.03%,6.25%,3.52%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
3,3.91%,4.69%,68.75%,16.41%,3.12%,3.12%,0.00%,0.00%,0.00%,0.00%
4,0.39%,0.78%,18.75%,48.83%,17.19%,14.06%,0.00%,0.00%,0.00%,0.00%
5,0.00%,0.00%,6.25%,21.09%,43.75%,25.00%,2.34%,0.78%,0.00%,0.78%
6,0.00%,0.00%,0.00%,6.64%,24.61%,49.22%,7.81%,6.25%,0.00%,5.47%
7,0.00%,0.00%,0.00%,3.12%,10.16%,7.81%,38.28%,25.78%,4.10%,10.74%
8,0.00%,0.00%,0.00%,0.39%,1.17%,0.78%,23.44%,31.25%,20.51%,22.46%
9,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,17.19%,23.44%,34.18%,25.20%
10,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,10.94%,12.50%,41.21%,35.35%


In [27]:
team_points2 = {k:pd.DataFrame(v, index = [k]) for k,v in team_points.items()}
perc = pd.concat(team_points2.values()).T.sort_index().T.rename_axis('RANK')
perc.loc[:,'Top 4'] = perc.iloc[:,:4].sum(axis=1)
perc

,1,2,3,4,5,6,7,8,9,10,Top 4
RANK,,,,,,,,,,,
KKR,0.625,0.332031,0.039062,0.003906,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
RR,0.375,0.570312,0.046875,0.007812,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
SRH,NaN,0.062500,0.687500,0.187500,0.062500,NaN,NaN,NaN,NaN,NaN,0.937500
CSK,NaN,0.035156,0.164062,0.488281,0.210938,0.066406,0.031250,0.003906,NaN,NaN,0.687500
DC,NaN,NaN,0.031250,0.171875,0.437500,0.246094,0.101562,0.011719,NaN,NaN,0.203125
LSG,NaN,NaN,0.031250,0.140625,0.250000,0.492188,0.078125,0.007812,NaN,NaN,0.171875
RCB,NaN,NaN,NaN,NaN,0.023438,0.078125,0.382812,0.234375,0.171875,0.109375,0.000000
PBKS,NaN,NaN,NaN,NaN,0.007812,0.062500,0.257812,0.312500,0.234375,0.125000,0.000000
MI,NaN,NaN,NaN,NaN,NaN,NaN,0.041016,0.205078,0.341797,0.412109,0.000000


In [20]:
perc.T[['Top 4']].sort_values(by = 'Top 4', ascending = False).index



Index(['SRH', 'CSK', 'DC', 'LSG', 'KKR', 'RR', 'PBKS', 'GT', 'RCB', 'MI'], dtype='object', name='RANK')

In [13]:
def evaluate_condition(node, points_dict, check_team):
    # Example condition: Check if the leaf node's match result is a tie
    winning_pts = dict(Counter(node.winners))
    for t in points_dict.keys():
        if t not in winning_pts.keys():
            winning_pts[t] = 0
        else: winning_pts[t] = winning_pts[t]*2

            
    all_points = list({t: points_dict[t] + winning_pts[t] for t in points_dict.keys()}.items())
    
    all_points = [i[0] for i in sorted(list(all_points), key = lambda x: -x[-1])]

    return all_points.index(check_team)+1
    # top4 = all_points[:3]

    # if all([t in [i[0] for i in top4] for t in check_team]): 
    #     print(all_points)
    #     return True
    # return False

def prune_tree(root, points_dict, check_team):
    # print(check_team)
    point_list = []
    stack = [root]
    leaf_node = []
    leaf_scenes = []
    while stack:
        node = stack.pop()
        if not node.children:  # Leaf node
            leaf_node.append(node)
            rank = evaluate_condition(node, dict(points_dict), check_team)
            point_list.append(rank)

            leaf_scene = pd.DataFrame(data = list(node.winners)).T
            leaf_scenes.append(leaf_scene)
            # if not evaluate_condition(node, dict(points_dict), check_team):
            #     # Prune subtree by removing all children
            #     node.parent.children.remove(node)
            # else:
            #     win_list.append(node)
        else:
            stack.extend(node.children)

    point_dict = dict(Counter(point_list))
    point_dict = {k:v/len(point_list) for k,v in point_dict.items()}
    return leaf_scenes, point_list, point_dict

points_dict = points_table.set_index('TEAM_NAME').to_dict()['PTS']



team_points = {}
scenario = {}
# for t in mapping['Team Abbr'].unique():
for t in teams:
    print(t)
    leaf_scenes, point_list, point_dict = prune_tree(copy.deepcopy(root), points_dict, t)
    team_points[t] =  point_dict

    # scenario[t] = pd.DataFrame(columns = [f"{m.team1.name} v {m.team2.name}" for m in match_dict.values()])
    scenario[t] = pd.concat(leaf_scenes).reset_index(drop = True)
    scenario[t] = scenario[t].reset_index(drop=True).rename_axis('ID').reset_index().merge(pd.DataFrame(point_list).reset_index(drop=True).rename_axis('ID').reset_index(), on = 'ID', how = 'left')
    scenario[t].columns = ['ID'] + [f"{m.team1.name} v {m.team2.name}" for m in match_dict.values()] + ['RANK']

   

KKR
RR
SRH
CSK
DC
LSG
RCB
PBKS
MI
GT


In [14]:
# display(scenario[t][scenario[t]['RANK']>4])
t = 'RCB'
display(scenario[t])
match_prob = {}
a = scenario[t][scenario[t]['RANK']<=4].copy()
for m in [f"{m.team1.name} v {m.team2.name}" for m in match_dict.values()]:
    v = a[m].value_counts(normalize = True).sort_values(ascending = True).rename_axis('MATCH').reset_index().T.reset_index()
    v.columns = v.iloc[0]
    v = v.iloc[1:,:]
    v = v.melt(id_vars = 'MATCH', var_name = 'WIN_TEAM', value_name = 'REQD_PERC').sort_values(by = 'REQD_PERC',ascending = False).iloc[:1,:]
    # .reset_index()
    match_prob[m] = v

final_scenario = pd.concat(match_prob.values()).reset_index(drop = True)


final_scenario['REQD_PERC'] = final_scenario['REQD_PERC'].fillna(0).apply(lambda x: f'{x:.2%}').T

# Define a function to apply conditional formatting
def highlight_cells(val):
    return f'background-color: rgba(0, 100, 255, {val})'  # Green color with alpha based on value

# Apply styling to the DataFrame
# final_scenario = final_scenario.style.apply(highlight_cells)

# Render styled DataFrame
final_scenario

,ID,PBKS v RCB,GT v CSK,KKR v MI,CSK v RR,RCB v DC,GT v KKR,DC v LSG,RR v PBKS,SRH v GT,MI v LSG,RCB v CSK,SRH v PBKS,RR v KKR,RANK
0,0,RCB,CSK,MI,RR,DC,KKR,LSG,PBKS,GT,LSG,CSK,PBKS,KKR,8
1,1,RCB,CSK,MI,RR,DC,KKR,LSG,PBKS,GT,LSG,CSK,PBKS,RR,8
2,2,RCB,CSK,MI,RR,DC,KKR,LSG,PBKS,GT,LSG,CSK,SRH,KKR,7
3,3,RCB,CSK,MI,RR,DC,KKR,LSG,PBKS,GT,LSG,CSK,SRH,RR,7
4,4,RCB,CSK,MI,RR,DC,KKR,LSG,PBKS,GT,LSG,RCB,PBKS,KKR,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,8187,PBKS,GT,KKR,CSK,RCB,GT,DC,RR,SRH,MI,CSK,SRH,RR,8
8188,8188,PBKS,GT,KKR,CSK,RCB,GT,DC,RR,SRH,MI,RCB,PBKS,KKR,7
8189,8189,PBKS,GT,KKR,CSK,RCB,GT,DC,RR,SRH,MI,RCB,PBKS,RR,7
8190,8190,PBKS,GT,KKR,CSK,RCB,GT,DC,RR,SRH,MI,RCB,SRH,KKR,7


,MATCH,WIN_TEAM,REQD_PERC
